In [2]:
import PyPDF2
import re
from os import listdir
from os.path import isfile, join, exists
import sys
import pandas as pd

reg_exp1='\d+[,.]?\d*' #number 
reg_exp2=r'\S+' #word
reg_exp3=r'.*' #line
reg_exp4=r'\D+' #non digits only

def find_exp(exp,text):
    match=re.search(exp,text)
    return re.findall(reg_exp1,text[match.end():])
def find_ocurrence(exp,text,num):
    loc=re.finditer(exp,text)
    k=0
    temp=-1
    for i in loc:
        k+=1
        if k==num:
            return i.start()
    return -1
def to_excel(filename,dictionary):
    df = pd.DataFrame(dictionary)
    df.to_excel(filename, index=True, header=True)
def read_pdf(filename):
    output=""
    if exists(filename) and isfile(filename) and filename.endswith('.pdf'):
        pdf1=open(filename,'rb')
        pd_read=PyPDF2.PdfFileReader(pdf1)
        pages=pd_read.numPages
        for i in range(0,pages):
            page1=pd_read.getPage(i)
            output+=page1.extractText()
        pdf1.close()
    return output
def read_digit(text,location): # reads the numbers befor a location in the text and returns the number (integer)
    if text=='' or location==-1:
        return 0
    temp=location-1
    number=0
    dig_counter=0
    while temp>=0 and not text[temp].isdigit():
        temp-=1
    while temp>=0 and text[temp].isdigit():
        number=int(text[temp])*(10**dig_counter)+number
        temp-=1
        dig_counter+=1
    return number
    
mypath='/Users/yoavbrezinov/Desktop/McGill/LDI/Gabi/GPT_MODEL/TESTING/FINAL_TEST/'
allfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
all_data={}
data={'characters':0, 'words':0, 'sentences':0,'Score':0, 'Issues left':0, 
      'Critical':0, 'Unique Words':0, 'Rare Words':0, 'Word Length':0, 'Sentence Length':0, 'ERROR':''}
total_files=0
pdf_files=0
no_crit_err=0
bed_pdfs=0
for f in allfiles:
    total_files+=1
    if f.endswith('.pdf'):
        pdf_files+=1
        all_data[f]={}
        all_data[f]['ERROR']=''
        output=read_pdf(join(mypath,f))
        nums=find_exp('General metrics',output)
        all_data[f]['characters']=int(nums[0].replace(",",""))
        all_data[f]['words']=int(nums[1].replace(",",""))
        all_data[f]['sentences']=int(nums[2].replace(",",""))
        nums=find_exp('Score Writing Issues',output)
        all_data[f]['Score']=int(nums[0].replace(",",""))
        nums=find_exp('by Grammarly',output)
        all_data[f]['Issues left']=int(nums[0].replace(",",""))
        temp3=find_ocurrence('Critical',output,1)
        if temp3!=-1 and not output[temp3-2].isdigit():
            all_data[f]['Critical']=0
        else:
            all_data[f]['Critical']=int(nums[1].replace(",",""))
        nums=find_exp('Unique Words',output)
        all_data[f]['Unique Words']=int(nums[0].replace(",",""))
        nums=find_exp('Rare Words',output)
        all_data[f]['Rare Words']=int(nums[0].replace(",",""))
        nums=find_exp('Word Length',output)
        all_data[f]['Word Length']=float(nums[0].replace(",",""))
        all_data[f]['Sentence Length']=float(nums[1].replace(",",""))
        #Writing Issues#
        if all_data[f]['Critical']==0:
            loc=find_ocurrence('Clarity',output,1)
            if loc!=-1:
                dig=read_digit(output,loc)
                all_data[f]['Clarity']=dig
                all_data[f]['Wordy sentences']=dig
            no_crit_err+=1
            print("No critical errors in: ",f)
            all_data[f]['ERROR']='No critical mistakes'
            continue
        temp1=find_ocurrence('Writing Issues',output,2)
        if not output[temp1+len('Writing Issues')+1].isdigit():
            bed_pdfs+=1
            print("Error with PDF: ",f)
            all_data[f]['ERROR']='Error with PDF'
            bed_text='Measures vocabulary diversity by calculating the\npercentage of words used only once in your\ndocumentunique words'
            output=output.replace(bed_text,'')
        temp2=find_ocurrence('Report',output,3)
        words=re.findall(reg_exp3,output[temp1:temp2])
        del words[0]
        for word in words:
            if word!='' and word[0].isdigit():
                num=re.findall(reg_exp1,word)
                expr=re.findall(reg_exp4,word)
                all_data[f][expr[0][1:]]=int(num[0])
        if all_data[f]['ERROR']=='':
            all_data[f]['ERROR']='Process OK'
print(all_data)
print('#'*100)
print("Punctuation in compound/complex - add -> sentences")
print(f'Total files in {mypath} --> {total_files}\nPDF files: {pdf_files}\nPDFs without Critical mistakes: {no_crit_err}\nBed PDFs: {bed_pdfs}\n')
to_excel(mypath+'/25TEST_ORIGINAL.xlsx',all_data)

Error with PDF:  231.pdf
{'4512.pdf': {'ERROR': 'Process OK', 'characters': 1604, 'words': 256, 'sentences': 14, 'Score': 75, 'Issues left': 18, 'Critical': 8, 'Unique Words': 41, 'Rare Words': 36, 'Word Length': 4.6, 'Sentence Length': 18.3, 'Correctness': 8, 'Misuse of modi\x00ers': 1, 'Wrong or missing prepositions': 1, 'Comma misuse within clauses': 1, 'Confused words': 1, 'Incorrect verb forms': 1, 'Determiner use (a/an/the/this, etc.)': 2, 'Incorrect noun number': 1}, '111.pdf': {'ERROR': 'Process OK', 'characters': 1712, 'words': 243, 'sentences': 14, 'Score': 79, 'Issues left': 18, 'Critical': 1, 'Unique Words': 46, 'Rare Words': 44, 'Word Length': 5.9, 'Sentence Length': 17.4, 'Clarity': 1, 'Wordy sentences': 1, 'Correctness': 1, 'Incorrect noun number': 1}, '321.pdf': {'ERROR': 'Process OK', 'characters': 2166, 'words': 306, 'sentences': 15, 'Score': 86, 'Issues left': 15, 'Critical': 1, 'Unique Words': 41, 'Rare Words': 38, 'Word Length': 5.9, 'Sentence Length': 20.4, 'Clari

In [ ]:
#print(read_pdf(mypath+'4.pdf'))